In [1]:
import networkx as nx
from tqdm import tqdm
import random
from leafer import Leafer
import numpy as np

In [2]:
G = nx.read_edgelist("../Data/IsA_Graph2.edgelist", delimiter=" ", create_using=nx.DiGraph)

In [3]:
l = Leafer(G)
#iterator = l.leafs_generator()

In [7]:
train, test = l.split_train_test(generation_depth=50, # до какого уровня в топ. сортировке идти
                                 p=0.1, # вероятность что подходящий случай уйдет в тест
                                 p_divide_leafs=0.5, 
                                 # вероятность что листья поделим пополам трейн-тест
                                 # а не засунем целый случай в трейн или в тест
                                 min_to_test_rate=0.5) 
                                #минимальное количество доли вершин которых не было в 
                                #трейне чтобы поделить пополам на трейн-тест
                                #то есть если 6\10 вершин были трейне то значит все 10 в трейн
                                #если 5\10 были в трейне, то значит оставшиеся можем кинуть в тест
                                

In [8]:
test

[{'children': 'benzene_formula',
  'parents': 'benzene',
  'grandparents': ['tangible_thing',
   'chemical_compound',
   'aromatic_hydrocarbon',
   'voc'],
  'case': 'only_child_leaf'},
 {'children': 'egg_sandwich',
  'parents': 'breakfast_sandwich',
  'grandparents': ['tangible_thing', 'sandwich'],
  'case': 'only_child_leaf'},
 {'children': 'ministerial_department',
  'parents': 'british_government_department',
  'grandparents': ['tangible_thing', 'legal_government_organization'],
  'case': 'only_child_leaf'},
 {'children': 'chlorophyll_b_molecule',
  'parents': 'chlorophyll_b',
  'grandparents': ['tangible_thing',
   'photosynthetic_accessory_pigment',
   'chlorophyll'],
  'case': 'only_child_leaf'},
 {'children': 'gaseous_chloropicrin',
  'parents': 'chloropicrin',
  'grandparents': ['tangible_thing',
   'chemical_compound',
   'compound',
   'choking_agent'],
  'case': 'only_child_leaf'},
 {'children': 'turkey_club',
  'parents': 'club_sandwich',
  'grandparents': ['tangible_thing